## 分车型预测-LSTM

In [1]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, RNN, GRU, SimpleRNN
from sklearn.model_selection import train_test_split
from keras.initializers import glorot_uniform

seed = 2019
random.seed(seed)
tf.set_random_seed(seed)
np.random.seed(seed)
warnings.filterwarnings('ignore')

C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

In [2]:
data_path = './data/'
train_user_reply_data = pd.read_csv(data_path + 'train_user_reply_data.csv', encoding='utf-8')
train_search_data = pd.read_csv(data_path + 'train_search_data.csv', encoding='utf-8')
train_sales_data = pd.read_csv(data_path + 'train_sales_data.csv', encoding='utf-8')
evaluation_public = pd.read_csv(data_path + 'evaluation_public.csv', encoding='utf-8')

train_sales_data = train_sales_data.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True)
evaluation_public = evaluation_public.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True)

In [3]:
# train_sales_data.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True).to_csv('temp.csv', encoding='utf-8', index=None, sep='\t')
# evaluation_public.sort_values(['regYear', 'regMonth', 'model', 'province'], ascending=True).to_csv('temp_sub.csv', encoding='utf-8', index=None, sep='\t')

In [4]:
cars = ['a50a6db3d691e9ce', 'b135f55ced5c4c67', '9011c3cf495e3c7b', '28e29f2c03dcd84c', '54fc07138d70374c', 'd0f245b8781e3631', 'ffeb3a4cec627e8c', 'a7128d308ae14d73', '7aab7fca2470987e', '42d4439853da7221', '2509a94e62fd3e4b', '9e967965ad4a6c07', '346393c2c6305fb1', '7a7885e2d7c00bcf', '17bc272c93f19d56', 'f8a6975573af1b33', 'feabbf46658382b9', 'fde95ea242abd896', 'd4efbebb087fd03f', '04e66e578f653ab9', '7245e0ee27b195cd', 'f270f6a489c6a9d7', '02aab221aabc03b9', 'b4be3a4917289c82', 'f5d69960089c3614', 'c06a2a387c0ee510', '6858d6dfe680bdf7', '12f8b7e14947c34d', '07a30393ec51530e', 'cc21c7e91a3b5a0c', '3c974920a76ac9c1', '37aa9169b575ef79', 'ea489c253676aafc', '97f15de12cfabbd5', 'fc32b1a017b34efe', 'b25c4e2e3856af22', '854982e5264fb53e', '8527c11fccaa94e2', '283d331fdf05a655', '9a390098bf87b814', 'a28bb927b6fcb33c', 'a9a43d1a7ecbe75d', 'ef76a85c4b39f693', '4a103c30d593fbbe', '0797526c057dcf5b', 'c6cd4e0e073f5ac2', '1181894ae2811540', 'af6f4f548684e14d', '6155b214590c66e6', '5b1c11c3efed5312', '936168bd4850913d', 'c6833cb891626c17', '8cbac8dd8f0e89e2', 'a432c483b5beb856', 'cd5841d44fd7625e', '2a2ab41f8f6ff1cb', '7023efdab9cedc03', 'bb9fbec9a2833839', '79de4e4b24c35b04', '4f79773e600518a6', '28956642949e0e57', '3d7554f1f56dd664', 'b9aa8592ab2492da', '17363f08d683d52b', 'bc5bbd459c18fefd', '0aa15031db420212', '3e21824be728cbec', '7cf283430b3b5e38', '212083a9246d2fd3', 'da457d15788fe8ee', 'a207df29ec9583f0', '2d0d2c3403909fdb', '8c915fe4632fb9fa', '06880909932890ca', 'dff803b4024d261d', '63065128401bb3ff', '9c1c7ee8ebdda299', 'e8a33f68c15dcf7c', '32d3069d17aa47c2', '5f727f32393ade77', '5d7fb682edd0f937', '61e73e32ad101892']
provinces = ['浙江', '福建', '四川', '陕西', '安徽', '湖南', '广东', '云南', '上海', '山东', '湖北', '黑龙江', '江苏', '广西', '内蒙古', '辽宁', '北京', '重庆', '河北', '山西', '江西', '河南']

# Normalization
train_sales_data['salesVolume'] = train_sales_data['salesVolume'].map(lambda index: np.log2(index) + 1)
scaler = MinMaxScaler(feature_range=(0, 1))
train_sales_data[['salesVolume']] = scaler.fit_transform(train_sales_data[['salesVolume']])
train_sales_data.head(3)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
506,上海,310000,02aab221aabc03b9,Sedan,2016,1,0.666707
507,云南,530000,02aab221aabc03b9,Sedan,2016,1,0.567856
508,内蒙古,150000,02aab221aabc03b9,Sedan,2016,1,0.469225


In [5]:
def generate_batch(data_set, look_back, gap_days):
    data_x, data_y = [], []
    for i in range(len(data_set) - look_back - gap_days + 1):
        a = data_set[i:(i + look_back), 0]
        data_x.append(a)
        data_y.append([data_set[i + look_back + gap_days - 1, 0]])
    return np.array(data_x), np.array(data_y), data_set[-look_back:, 0].reshape(1, look_back)

In [6]:
def metrics(data):
    data = scaler.inverse_transform(data)
    data = 2 ** (data - 1)
    print(data)
    
    a = data[:, 0]
    b = data[:, 1]
    res = np.sqrt(np.sum((a - b) ** 2) / len(a)) / np.mean(a)
    print('validate rmse:', 1 - res)
    return (1 - res)

In [7]:
from keras.layers.advanced_activations import PReLU

def construct_lstm_model(x_train, x_test, y_train, y_test, test, car_name, batch_size):
    print('batch_size:', batch_size)
#     y_train = scaler.inverse_transform(y_train)
#     y_test = scaler.inverse_transform(y_test)
    model = Sequential()
    model.add(LSTM(32, input_shape=(1, look_back), return_sequences=True, kernel_initializer=glorot_uniform(seed=seed)))
    model.add(LSTM(32, return_sequences=False, kernel_initializer=glorot_uniform(seed=seed)))
    model.add(Dense(units=1, kernel_initializer=glorot_uniform(seed=seed)))
    model.add(PReLU())
    model.compile(loss='mean_squared_error', optimizer='adam')   # loss='mse'  'mean_squared_error'
#     model.summary()
    early_stopping = EarlyStopping('val_loss', patience=20, verbose=1)
    best_model_path = './model/' + car_name + '.h5'
    checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', mode='min', save_best_only=True, verbose=0, save_weights_only=False)
    callbacks = [checkpoint, early_stopping]
    model.fit(x_train, y_train, epochs=500, batch_size=batch_size, verbose=0, shuffle=False, callbacks=callbacks, validation_data=(x_test, y_test))
    model.load_weights(best_model_path)     # load best model parameters   shuffle=False
    predict = model.predict(test)
    predict = scaler.inverse_transform(predict)
    predict = 2 ** (predict - 1)
#     print(np.hstack((y_test, model.predict(x_test))))
    rmse = metrics(np.hstack((y_test, model.predict(x_test))))
    return predict, rmse

In [8]:
file_name = os.listdir('./model/')
for file in file_name:
    os.remove('./model/' + file)
print(os.listdir('./model/'))

look_back = 10
label_length = 4
batch_size = 16
submit = pd.DataFrame()
flag = 0
all_mse = []
thresh = 1.32

for car in cars:
    labels = np.zeros([len(provinces), label_length])
    rmse_total = 0
    for k in range(1, 5, 1):
        print('{} car:{}'.format(flag+1, car))
        train = np.array([[0 for _ in range(look_back)]])
        label = np.array([[0]])
        test = np.array([[0 for _ in range(look_back)]])

        for province in provinces: 
            temp = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
    #         print(temp[['province', 'model', 'regYear', 'regMonth', 'salesVolume']])
            data = temp['salesVolume'].values.reshape(-1, 1).astype('float32')
            train_data, train_label, test_data = generate_batch(data, look_back, k)
            train = np.vstack((train, train_data))
            label = np.vstack((label, train_label))
            test = np.vstack((test, test_data))

        train = train[1:]
        label = label[1:]
        test = test[1:]
        train = np.reshape(train, (-1, 1, look_back))
        test = np.reshape(test, (-1, 1, look_back))
        x_train,x_test,y_train,y_test = train_test_split(train, label, test_size=0.1, random_state=seed)

        print('train shape:', train.shape)
        print('label shape:', label.shape)
        print('test shape:', test.shape)
        res, rmse = construct_lstm_model(x_train, x_test, y_train, y_test, test, car, int((train.shape[0] * 0.9) / 22))
        K.clear_session()    #  清除tf session
        labels[:, k-1] = res.reshape(1, -1)
        rmse_total += rmse
    print(labels)
    print('average rmse:', rmse_total / 4)
    flag += 1
    all_mse.append(rmse_total / 4)

    index = 0
    submit_partial = pd.DataFrame()
    for province_2 in provinces: 
        temp = evaluation_public[(evaluation_public['model'] == car) & (evaluation_public['province'] == province_2)]
        temp['forecastVolum'] = labels[index]
        submit_partial = pd.concat([submit_partial, temp], axis=0, ignore_index=True)
        index += 1
    submit = pd.concat([submit, submit_partial], axis=0)

print(pd.DataFrame({'car': cars, 'rmse': all_mse}))
print('all validation rmse:', np.mean(all_mse))

[]
1 car:a50a6db3d691e9ce
train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00041: early stopping
[[505.99990097 342.79727852]
 [205.00005415 179.20088912]
 [277.00001148 235.07666391]
 [186.99997475 243.64785287]
 [ 69.9999972   68.95579381]
 [108.00001109 128.24203909]
 [438.99998416 309.71591034]
 [456.99998136 320.39880458]
 [129.00003593 145.28597355]
 [215.9999768  152.13418054]
 [291.00005581 275.43185435]
 [146.99999695 147.28786678]
 [155.99997376 253.46859018]
 [326.00008753 315.80994703]
 [469.00003486 527.90894647]
 [279.0000113  244.06876477]
 [250.9999714  183.44335782]
 [179.99997496 203.35032982]
 [174.00002648  96.54247529]
 [245.00000606 310.50464845]
 [208.99994294 202.88342089]
 [ 76.00000245  70.91371466]
 [ 45.99999794  66.20616636]
 [145.99997893 159.69730308]
 [146.99999695 245.18527193]
 [285.99997371 317.27668898]
 [551.00014181 364.20589327]
 [246.99996538 172.06237936]
 [ 96.00000781 104.78751344]
 [263.99994088 235

Epoch 00469: early stopping
[[ 146.99999695  156.21355972]
 [ 233.00000251  176.46952318]
 [1020.00018486 1005.26667509]
 [ 371.00004081  317.29710069]
 [  76.00000245  118.77146172]
 [ 179.99997496   92.84935361]
 [ 384.00009048  368.65592032]
 [ 407.99997673  498.63822427]
 [ 233.00000251  168.10580675]
 [ 308.00004471  269.22600538]
 [ 121.99999587  106.35849568]
 [  93.00000537   81.06236305]
 [ 464.00005273  307.41907987]
 [ 145.00001747  137.69701221]
 [1803.00025245 1763.32704259]
 [ 182.99997653  193.62428016]
 [ 120.99999821  127.13332093]
 [  69.9999972    84.22944674]
 [ 336.00004907  196.27272158]
 [  67.99999205  126.79246211]
 [ 479.00013488  512.38134311]
 [  73.0000048   117.5403483 ]
 [ 195.99997326  245.49896613]
 [  92.00000298  108.79394764]
 [  78.00000326   79.90994389]]
validate rmse: 0.8095170671323608
[[ 524.70202637  360.34442139  465.32901001  430.33694458]
 [ 244.64686584  214.09472656  244.51881409  216.52122498]
 [ 611.76428223  392.9095459   519.2980957  

Epoch 00096: early stopping
[[1001.99976533 1277.45470867]
 [ 556.0001412   428.41250051]
 [ 461.00000027  476.32561914]
 [ 164.99997691  266.84337788]
 [ 311.99997159  344.95716063]
 [1631.00024893 1332.97407889]
 [1131.00006436 1153.06972587]
 [ 182.00003656  253.56309611]
 [1283.99969223 1253.76386034]
 [ 140.99998243  138.93855014]
 [ 650.99994295  656.59616337]
 [ 860.99981585  796.70371331]
 [ 680.99984896  658.90948391]
 [1262.99983346 1777.21587785]
 [ 388.999972    444.87183981]
 [ 272.99995068  341.1559207 ]
 [ 705.00013454 1227.64399554]
 [ 425.00003076  450.82016302]
 [ 140.99998243  137.89559766]
 [ 451.99991752  431.00956516]
 [ 260.99994013  339.90643703]
 [  74.00000482  129.51492166]
 [ 856.00012144  761.74979902]
 [ 150.9999799   135.54607106]
 [1244.00006883 1268.58366178]
 [1111.99972946  666.8817981 ]
 [ 426.99989052  395.7114465 ]
 [ 737.99999366  797.07081005]
 [1590.00008955 1564.30554476]]
validate rmse: 0.722800178026604
4 car:28e29f2c03dcd84c
train shape: (26

train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00139: early stopping
[[ 755.99996772  351.81129114]
 [ 104.99998589   88.72521414]
 [ 909.99994715  712.38837274]
 [ 453.000062    431.49755205]
 [ 243.99994048  111.79004256]
 [  68.9999904    63.56315048]
 [ 414.99992771  481.39371187]
 [ 777.0000875   327.97048146]
 [  21.00000262   18.75125796]
 [ 201.00004069  130.78946973]
 [ 362.9999884   295.92263078]
 [  44.99999972   20.50943136]
 [ 515.99992695  406.81330474]
 [  66.99999548   58.91355911]
 [ 169.9999718   133.22731691]
 [ 532.99996301  528.91115314]
 [ 229.00005252  108.27116238]
 [ 136.00003364  103.1883055 ]
 [ 325.00001247  338.13380615]
 [  21.00000262   29.67747236]
 [ 134.99999684   92.39546438]
 [  82.00000212   85.38464796]
 [  65.00000066   82.1140168 ]
 [ 112.9999944   123.09180063]
 [2477.99970042  387.50825647]
 [  26.99999861   39.53957059]
 [  76.00000245   51.50179893]
 [ 314.99995226  164.98907816]
 [ 133.00000263  

Epoch 00304: early stopping
[[ 644.00011222  696.29291217]
 [ 239.99993885  319.44912251]
 [ 879.99991082  759.30174516]
 [ 252.00006595  232.66098118]
 [ 157.00002479  141.85342989]
 [ 355.00009949  292.15835364]
 [ 507.00011852  541.33110716]
 [ 317.9999176   169.52120133]
 [ 207.99994095  190.50109364]
 [ 501.00013178  543.07515725]
 [ 199.00001997  127.07717748]
 [ 118.00000492  100.43090885]
 [1856.99967752 1890.70831557]
 [ 472.0000925   391.7690285 ]
 [1397.00031787 1192.69886122]
 [ 909.99994715 1027.19073807]
 [ 459.00011542  363.36296797]
 [ 220.99999952  190.25592622]
 [ 488.99994471  608.48263988]
 [ 308.99998517  385.43579175]
 [ 315.99992899  219.32920417]
 [ 121.99999587  230.9757802 ]
 [ 888.00017958  800.05365926]
 [ 109.99999498  124.29289387]
 [ 104.99998589   99.73887012]]
validate rmse: 0.82304165523959
[[1043.40075684 1061.8170166   832.50543213  976.26403809]
 [ 246.96504211  280.07254028  185.00601196  176.04666138]
 [ 674.27172852  729.81494141  592.73114014  6

...

76 car:63065128401bb3ff
train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00024: early stopping
[[2533.00006227 1760.76843346]
 [1810.00017269 1344.95646056]
 [2119.00036708 1900.39527728]
 [1800.99948853 1795.71175361]
 [ 826.99994548  644.32090343]
 [ 882.99989861 1933.72392423]
 [4988.99985057 2824.44431281]
 [7066.00146145 2881.83067288]
 [ 275.99992493  485.25982185]
 [2108.00051678 1107.57961797]
 [2723.0001902  2880.96673377]
 [ 466.00004098  523.16967922]
 [1398.99979179 1849.5130744 ]
 [1235.00021645 1123.01831658]
 [1358.99977231 1933.89276071]
 [2053.00029252 1803.20842631]
 [2760.99926852 1745.65096708]
 [ 365.9999813   620.61923372]
 [1576.99994121 1271.23658946]
 [ 976.99998808  768.49888357]
 [ 407.00005056  603.73848158]
 [ 749.00003914  676.63678654]
 [ 413.00004393  611.13944355]
 [1307.0000319  1048.16268788]
 [1756.0002076   952.71861932]
 [ 586.99985925  796.20917836]
 [2455.00049599 1230.03961018]
 [2357.00053167 1590.420

validate rmse: -0.002980283591964783
77 car:9c1c7ee8ebdda299
train shape: (242, 1, 10)
label shape: (242, 1)
test shape: (22, 1, 10)
batch_size: 9
Epoch 00024: early stopping
[[ 203.99997262  152.06463811]
 [ 274.00002607  123.56922761]
 [1114.00009728  265.8866808 ]
 [ 432.9998972   393.08947135]
 [ 136.00003364  197.08600071]
 [ 249.99996866  370.54471287]
 [ 635.00007591  554.84142497]
 [ 781.99997747  632.19415039]
 [ 234.00000576   96.21900454]
 [ 509.00005687  634.70579917]
 [ 283.00003144  344.90109127]
 [ 400.99991886  360.99265449]
 [ 900.99998566  176.23846582]
 [ 201.00004069   89.85617562]
 [1908.99992917  260.98000198]
 [ 312.99995326  102.36234348]
 [  46.99999495  106.11130133]
 [  52.00000708  132.58186776]
 [ 177.00004151  364.31887738]
 [ 196.99998379  152.05546718]
 [ 851.99984214  568.5530571 ]
 [  34.00000316   46.34546281]
 [ 298.99995814  199.81633614]
 [  65.99999399  122.70026995]
 [ 176.00002774  205.48163025]]
validate rmse: 0.013677522730861869
[[ 695.966003

Epoch 00033: early stopping
[[106.0000048  283.5576502 ]
 [ 54.00000138  51.23886469]
 [ 38.99999862  38.31636898]
 [ 99.00001008 132.4473717 ]
 [ 18.99999768  53.43227425]
 [142.00000596 213.94272422]
 [298.99995814 237.71164525]
 [ 88.00000708  71.62251426]
 [134.00000129 205.69088101]
 [ 13.99999905  31.82724337]
 [122.99999157 223.38787178]
 [ 56.99999206  57.29212776]
 [134.99999684 217.44573014]
 [290.00005731 206.79674953]
 [588.99997224 384.48012358]
 [111.99998612  87.58737992]
 [185.000003   250.89762967]
 [ 33.00000393  37.79247986]
 [ 80.00000397 124.54402219]
 [ 34.9999959   36.76505789]
 [ 43.00000037  80.42528531]
 [120.99999821 133.68495454]
 [109.99999498  83.17835851]
 [ 32.00000315  30.32508011]
 [210.0000483  206.93434559]
 [223.99998952 173.56690755]
 [483.99992854 410.67918083]
 [ 63.99999286  60.11434832]
 [358.99992633 227.22256845]]
validate rmse: 0.5148688361118774
79 car:32d3069d17aa47c2
train shape: (264, 1, 10)
label shape: (264, 1)
test shape: (22, 1, 10)


train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00055: early stopping
[[190.99994644 182.09812111]
 [126.99997512 134.4514116 ]
 [301.00004527 359.02652804]
 [287.99993575 341.48979982]
 [ 86.00000737 102.35417113]
 [287.00002598 278.8473294 ]
 [214.99994616 185.16180561]
 [263.99994088 189.1014939 ]
 [ 78.00000326  71.90774538]
 [190.0000514  160.6138195 ]
 [193.9999998  184.7735895 ]
 [ 52.99999831  74.62633916]
 [351.00007632 353.0783737 ]
 [283.00003144 228.35892411]
 [737.99999366 615.25542931]
 [298.00002557 338.95052227]
 [232.00000846 189.00734501]
 [102.99999683 112.39871646]
 [138.00003108 142.34519518]
 [129.00003593 138.99737914]
 [103.99999232 105.21742178]
 [102.99999683  97.28439367]
 [110.99999676 105.17192319]
 [168.99999388 157.37959854]
 [190.0000514  122.47353988]
 [124.99997469 144.89810998]
 [332.00007938 241.73709455]
 [138.00003108 136.45324363]
 [289.0000092  300.18665029]
 [270.99999632 329.56846734]
 [717.99990806 

82 car:61e73e32ad101892
train shape: (242, 1, 10)
label shape: (242, 1)
test shape: (22, 1, 10)
batch_size: 9
Epoch 00045: early stopping
[[ 543.99990604  606.57801076]
 [ 475.99987513  470.96963159]
 [ 668.99998704  486.99746194]
 [ 302.99993598  455.21403355]
 [ 381.00002823  428.68425592]
 [ 616.00013695  691.71187   ]
 [ 792.00003647  634.21709811]
 [ 848.99983586  712.20426057]
 [ 341.99997286  346.8564001 ]
 [ 656.99983187  686.49976753]
 [ 245.99993147  262.56192904]
 [ 219.00001062  246.80694883]
 [ 819.99987211  559.67114545]
 [ 315.99992899  268.2803358 ]
 [ 891.99987953  506.55940701]
 [ 234.00000576  205.72455579]
 [ 586.99985925  502.93769034]
 [ 230.00006226  265.35420967]
 [1547.00021605 1478.46502893]
 [ 338.00001384  606.0124498 ]
 [1094.00017903  691.30593366]
 [ 555.00015884  728.08933176]
 [ 425.00003076  597.14734885]
 [ 250.9999714   348.62529349]
 [ 397.0000238   508.22762742]]
validate rmse: 0.7048426252649295
[[ 671.09545898  572.83880615  440.21322632  560.727

```
all validation rmse: 0.6483414301351754
```

In [9]:
submit = submit.sort_values(by=['id'], ascending=True)
submit['forecastVolum'] = submit['forecastVolum'].map(lambda index: int(np.round(index)))
print('res store over')

res store over


In [10]:
sub = submit.sort_values(by=['id'], ascending=True)
print(len(sub))
sub['forecastVolum'].mean()
sub.groupby(['regMonth'])['forecastVolum'].mean()

7216


505.5597283813747

regMonth
1    578.380266
2    506.344235
3    477.557095
4    459.957317
Name: forecastVolum, dtype: float64

In [11]:
sub = submit.copy()
sub['forecastVolum'] = sub['forecastVolum'] / thresh
sub['forecastVolum'] = sub['forecastVolum'].map(lambda index: int(np.round(index)))
sub.groupby(['regMonth'])['forecastVolum'].mean()
sub.describe()
sub[['id', 'forecastVolum']].to_csv('./submit/fusai_lstm_1.csv', encoding='utf-8', index=None)

regMonth
1    438.177938
2    383.589246
3    361.787694
4    348.456208
Name: forecastVolum, dtype: float64

,id,adcode,regYear,regMonth,forecastVolum
count,7216.000000,7216.000000,7216.0,7216.000000,7216.000000
mean,3658.939024,347727.272727,2018.0,2.500000,383.002772
std,2114.152561,136288.963363,0.0,1.118111,508.446941
min,1.000000,110000.000000,2018.0,1.000000,1.000000
25%,1826.750000,230000.000000,2018.0,1.750000,98.000000
50%,3652.500000,355000.000000,2018.0,2.500000,227.000000
75%,5500.250000,440000.000000,2018.0,3.250000,461.000000
max,7304.000000,610000.000000,2018.0,4.000000,8486.000000
